In [36]:
import importlib
import os
import sys
import warnings

import numpy as np

from music21 import environment, stream

environment.set('midiPath', '/usr/bin/timidity')

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# Всё, что выводит CUDA / XLA, будет записано в файл
sys.stderr = open("xla_warnings.log", "w")



sys.path.append('src')

# в режиме редактирования сбрасуем кэшь иначе не видно изменений
for module in [
    "utils.music21_utils",
    "utils.data_utils",
    "utils.model_utils",
    "utils.generate_utils",
    "utils.train_utils",
    "utils.__init__"
    
]:
    imported_module = importlib.import_module(module)
    importlib.reload(imported_module)


from utils.__init__ import (
    split_parts,
    sequence_to_part,
    normalize_sequence, 
    denormalize_sequence, 
    prepare_decoder_inputs,
    build_triple_output_transformer,
    generate_full_autoregressive,
    train_model,
    get_midis_by_compositor,
    get_notes_multitrack,
    prepare_sequences,
)

## Загрузка midi по композитору

In [2]:
сhopin_midis = get_midis_by_compositor("Chopin")

In [3]:
# все ноты
notes = get_notes_multitrack(сhopin_midis)
print(f"Извлечено {len(notes)} нот из {len(сhopin_midis)} файлов")

Извлечено 57894 нот из 48 файлов


In [4]:
# --- Разделяем инструменты ---
melody_part, bass_part, chord_part = split_parts(notes)

In [5]:
# 2. Преобразование
# melody_X = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in melody_part])[None, ...]
chord_y = normalize_sequence([[n['pitch'], n['step'], n['duration']] for n in chord_part])[None, ...]
dec_inputs, dec_targets = prepare_decoder_inputs(chord_y)

In [6]:
# Преобразуем партии в массивы признаков
melody_X, melody_y = prepare_sequences(melody_part)
bass_X, bass_y = prepare_sequences(bass_part)
# chord_part — это список словарей
chord_seq = [[n['pitch'], float(n['step']), float(n['duration'])] for n in chord_part]
chord_X, chord_y = prepare_sequences(chord_seq)

In [9]:
# --- Синхронизируем длины ---
min_len = min(len(melody_X), len(bass_X), len(chord_X))
melody_X, melody_y = melody_X[:min_len], melody_y[:min_len]
bass_X, bass_y = bass_X[:min_len], bass_y[:min_len]
chord_X, chord_y = chord_X[:min_len], chord_y[:min_len]

In [10]:
# вариант: конкат по времени (length = 2 * L_enc)
# --- Входы и выходы для трансформера ---
enc_inputs = np.concatenate([melody_X, bass_X], axis=1)
dec_inputs = np.zeros_like(chord_y)
dec_inputs[:, 1:, :] = chord_y[:, :-1, :]  # shift right
dec_targets = chord_y

## Обучение

In [29]:
pitch_y = dec_targets[..., 0:1]
step_y = dec_targets[..., 1:2]
dur_y = dec_targets[..., 2:3]

In [30]:
# 3. Модель
#model = build_transformer()
model = build_triple_output_transformer()
#train_model(model, enc_inputs, dec_inputs, dec_targets, epochs=10)

In [31]:
history = model.fit(
[enc_inputs, dec_inputs],
{'pitch_out': pitch_y, 'step_out': step_y, 'duration_out': dur_y},
batch_size=64,
epochs=50,
validation_split=0.1
)

Epoch 1/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 153s 724ms/step - duration_out_loss: 0.0411 - loss: 2.6071 - pitch_out_loss: 2.5578 - step_out_loss: 0.0555 - val_duration_out_loss: 0.0139 - val_loss: 0.0403 - val_pitch_out_loss: 0.0041 - val_step_out_loss: 0.0587
Epoch 2/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - duration_out_loss: 0.0411 - loss: 0.0863 - pitch_out_loss: 0.0453 - step_out_loss: 0.0408 - val_duration_out_loss: 0.0139 - val_loss: 0.0162 - val_pitch_out_loss: 0.0062 - val_step_out_loss: 0.0059
Epoch 3/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - duration_out_loss: 0.0411 - loss: 0.0419 - pitch_out_loss: 0.0137 - step_out_loss: 0.0153 - val_duration_out_loss: 0.0139 - val_loss: 0.0135 - val_pitch_out_loss: 0.0054 - val_step_out_loss: 0.0022
Epoch 4/50
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - duration_out_loss: 0.0411 - loss: 0.0313 - pitch_out_loss: 0.0081 - step_out_loss: 0.0053 - val_duration_out_loss: 0.0139 - val_loss: 0.0127 - val_pitch_out_loss: 0.0039 - val_step_ou

## Генерация

In [13]:
F = 3
enc_input = np.expand_dims(enc_inputs[0], 0)
# (1, 64, 3)
seed_dec = np.zeros((1, 1, F))

In [32]:
generated = generate_full_autoregressive(model, enc_input, seed_dec, length=32)

In [80]:
generated_denorm = denormalize_sequence(generated)

In [62]:
# ==== Пример построения ====
melody_part = sequence_to_part(melody_X.reshape(melody_X.shape[0] * melody_X.shape[1], 3)[0:64], "Piano Right Hand")
bass_part = sequence_to_part(bass_X.reshape(bass_X.shape[0] * bass_X.shape[1], 3)[0:64], "Piano Left Hand")
chord_part = sequence_to_part(generated_denorm.reshape(generated_denorm.shape[0] * generated_denorm.shape[1], 3), "Tenor")
score = stream.Score()
score.insert(0, melody_part)
score.insert(0, bass_part)
score.insert(0, chord_part)
# Можно прослушать или сохранить:
score.write('midi', fp='three_voice_piece.mid')